# Dependencies

In [1]:
# %pip install fsutils --user
import pandas as pd
import numpy as np
import time
import logging
import itertools as it
import argparse
import os
import time
from pathlib import Path
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.spatial import distance
from pandas.api.types import CategoricalDtype
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from fsutils import run_sf_sql as rp, config, email, adls_gen2, log

# from utils.utils import worker_output_table_validation
from multiprocessing import Pool, freeze_support
from snowflake.connector.connection import SnowflakeConnection, SnowflakeCursor
from snowflake.connector.pandas_tools import write_pandas
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# from pandasql import sqldf
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import calinski_harabasz_score, silhouette_score
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

import os
import pandas as pd
import difflib

# Pandas Display Settings
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Set width to None to avoid wrapping

Inside config.py slfsvc-twa07
printing snowflake connection dic {'tenant': 'fabb61b8-3afe-4e75-b934-a47f782b8cd7', 'client_id': '5617df5a-e4aa-4898-8260-9ac51b67e1c0', 'akssecret': 'prod-fspe-sp-secret', 'vault': 'kv-rtlfspekv0701slfsvc07', 'sfsecretkey': 'sfkey', 'sfkeypass': 'sfpasskey'}
Initiating azcopy login
az login succeeded


# Utils

In [2]:
def get_last_datetime_token(filename: str) -> str:
    """
    Extract the date/time token at the end of filenames like:
    merged_clusters_PERSONAL_CLEANSING_20250212_1301.csv
    Returning '20250212_1301'.
    """
    if not filename.endswith(".csv"):
        return None  # skip anything that isn't a CSV
    no_ext = filename.split(".")[0]  # e.g., merged_clusters_PERSONAL_CLEANSING_20250212_1301
    tokens = no_ext.split("_")
    if len(tokens) < 3:
        return None  # skip anything that doesn't have at least two underscores
    return tokens[-2] + "_" + tokens[-1]  # e.g. 20250212_1301

In [3]:
def get_last_datetime_token(filename: str) -> str:
    """
    Extract the date/time token at the end of filenames like:
    merged_clusters_PERSONAL_CLEANSING_20250212_1301.csv
    Returning '20250212_1301'.
    """
    if not filename.endswith(".csv"):
        return None  # skip anything that isn't a CSV
    no_ext = filename.split(".")[0]
    tokens = no_ext.split("_")
    if len(tokens) < 3:
        return None
    return tokens[-2] + "_" + tokens[-1]

In [4]:
def normalize_str(s: str) -> str:
    """
    A helper to normalize the strings for matching.
    Example: 'HAND_&_BODY' -> 'HAND & BODY', 'ORAL_HYGIENE' -> 'ORAL HYGIENE', etc.
    Adjust as needed for your use-case.
    """
    # Replace `_&_` with ` & `
    s = s.replace("_&_", " & ")
    # Replace all remaining underscores with spaces
    s = s.replace("_", " ")
    # Strip and make uppercase (or whichever case you prefer)
    s = s.strip().upper()
    return s

In [5]:
def get_closest_match(query: str, valid_list: list) -> str:
    """
    Use difflib to get the closest match from valid_list to the query string.
    Returns None if there is no close match at all, else returns the best guess.
    """
    # difflib.get_close_matches(query, possibilities, n=1, cutoff=0.0)
    # will return a list of the single best match, or empty if none.
    matches = difflib.get_close_matches(query, valid_list, n=1, cutoff=0.0)
    return matches[0] if matches else None

In [6]:
# Finally, create a new column in df_filtered called "CAT_DSC"
def map_category_to_cat_dsc(category_value: str) -> str:
    # Step 1: Normalize the category_value
    norm_cat = normalize_str(category_value)
    # Step 2: Find the closest normalized official
    best_normed = get_closest_match(norm_cat, normalized_official)
    if best_normed is not None:
        # Return the original official category text
        return normalized_to_original[best_normed]
    else:
        # Fallback if no match at all
        return category_value  # or some sentinel like "UNKNOWN"

# Main

In [7]:
directory_path = "/home/jovyan/older code/notebooks/clustering_output/"


def get_last_datetime_token(filename: str) -> str:
    """
    Extract the date/time token at the end of filenames like:
    merged_clusters_PERSONAL_CLEANSING_20250212_1301.csv
    Returning '20250212_1301'.
    """
    if not filename.endswith(".csv"):
        return None  # skip anything that isn't a CSV
    no_ext = filename.split(".")[0]  # e.g., merged_clusters_PERSONAL_CLEANSING_20250212_1301
    tokens = no_ext.split("_")
    if len(tokens) < 3:
        return None  # skip anything that doesn't have at least two underscores
    return tokens[-2] + "_" + tokens[-1]  # e.g. 20250212_1301

In [8]:
all_files = os.listdir(directory_path)
# Filter down to only valid date/time tokens
valid_times = []
for f in all_files:
    dt = get_last_datetime_token(f)
    if dt is not None:
        valid_times.append(dt)
if not valid_times:
    raise ValueError("No valid CSV files found in directory.")

In [9]:
# Find the earliest (smallest) date/time across all files
earliest_datetime = min(valid_times)
print("earliest_datetime =", earliest_datetime)

earliest_datetime = 20250414_1639


In [10]:
# Rewrite all files to have the same end path as the earliest date/time
for file in all_files:
    dt = get_last_datetime_token(file)
    if dt is not None:
        new_filename = file.replace(dt, earliest_datetime)
        os.rename(os.path.join(directory_path, file), os.path.join(directory_path, new_filename))

In [11]:
# Now only read files that contain that earliest date/time
dfs = []
for file in os.listdir(directory_path):
    dt = get_last_datetime_token(file)
    if dt == earliest_datetime:
        df = pd.read_csv(os.path.join(directory_path, file))
        dfs.append(df)

# Concatenate the dataframes
dfs = pd.concat(dfs, ignore_index=True)

In [12]:
# Add a category column
dfs["category"] = dfs["external_granularity"].str.split("_").str[:-2].str.join("_")

print("Duplicates :", dfs["store_nbr"].nunique() * dfs["category"].nunique() < dfs.shape[0])
print(
    "1:1 mismatch :",
    dfs["store_nbr"].nunique() * dfs["category"].nunique()
    > dfs.drop_duplicates(subset=["store_nbr", "category"]).shape[0],
)

df_filtered = dfs[
    [
        "store_nbr",
        "category",
        "external_cluster_labels",
        "internal_cluster_labels",
        "demand_cluster_labels",
        "rebalanced_demand_cluster_labels",
    ]
].copy()

Duplicates : False
1:1 mismatch : False


In [13]:
df_filtered.category.unique()

array(['CANDY'], dtype=object)

In [14]:
df_filtered.nunique()

store_nbr                           6827
category                               1
external_cluster_labels                8
internal_cluster_labels                3
demand_cluster_labels                 24
rebalanced_demand_cluster_labels      15
dtype: int64

In [15]:
# ----------------------------------------------------------------------------
# 2) Manually define a dictionary that maps your existing category values
# to the new CAT_DSC strings you'd like.
# ----------------------------------------------------------------------------
category_mapping = {
    # Wave 1
    "VITAMINS": "VITAMINS",
    "ORAL_HYGIENE": "ORAL HYGIENE",
    "BEVERAGES": "BEVERAGES",
    "PERSONAL_CLEANSING": "PERSONAL CLEANSING",
    "HOUSEHOLD": "HOUSEHOLD",
    # Wave 2
    "HOME_DIAGNOSTICS": "HOME DIAGNOSTICS",
    "BABY_CARE": "BABY CARE",
    "ALLERGY_REMEDIES": "ALLERGY REMEDIES",
    "HAND_&_BODY": "HAND & BODY",
    "SNACKS": "SNACKS",
    "STATIONERY": "STATIONERY",
    "SHAVING_NEEDS": "SHAVING NEEDS",
    "FACIAL_CARE": "FACIAL CARE",
    "CHILDRENS_REMEDIES": "CHILDRENS REMEDIES",
    "ADULT_CARE": "ADULT CARE",
    "DIET_NUTRITION": "DIET/NUTRITION",
    "ACNE_HSC": "ACNE/HSC",
    "GROCERY": "GROCERY",
    "COLD_REMEDIES": "COLD REMEDIES",
    "DIGESTIVE_HEALTH": "DIGESTIVE HEALTH",
    "HAIR_CARE": "HAIR CARE",
    "EXTERNAL_PAIN": "EXTERNAL PAIN",
    "FIRST_AID": "FIRST AID",
    "CANDY": "CANDY",
    "DEODORANTS": "DEODORANTS",
    "TRIAL_TRAVEL": "TRIAL TRAVEL",
    "FEMININE_CARE": "FEMININE CARE",
    "HAIR_COLOR": "HAIR COLOR",
    "HOUSEHOLD_PAPER": "HOUSEHOLD PAPER",
    "TEXTURED_HAIR": "TEXTURED HAIR",
}

# ----------------------------------------------------------------------------
# 3) Apply the dictionary to create the CAT_DSC column
# ----------------------------------------------------------------------------
# If a given 'category' value is not in the dictionary, we can either default
# to the original string or set it to something like 'UNKNOWN'
df_filtered["CAT_DSC"] = df_filtered["category"].map(category_mapping).fillna(pd.NA)

In [16]:
missing_categories = df_filtered.loc[df_filtered["CAT_DSC"].isna(), "category"].unique()
assert len(missing_categories) == 0, f"Missing CAT_DSC mapping for categories: {missing_categories}"

In [17]:
# # Normalize the official CAT_DSC strings too, so we compare on the same basis.
# # We'll keep an extra structure mapping normalized -> original official.
# normalized_to_original = {}
# for cat in official_categories:
#     norm = normalize_str(cat)
#     normalized_to_original[norm] = cat  # store the *exact* original

# normalized_official = list(normalized_to_original.keys())

# df_filtered["CAT_DSC"] = df_filtered["category"].apply(map_category_to_cat_dsc)

# ###############################################################################
# # End result
# ###############################################################################
# print(df_filtered.head(10))


In [18]:
df_filtered["CAT_DSC"].value_counts()

CANDY    6827
Name: CAT_DSC, dtype: int64

In [19]:
df_filtered["category"].value_counts()

CANDY    6827
Name: category, dtype: int64

In [20]:
df_filtered["CAT_DSC"].unique()
# array(['BEVERAGES', 'VITAMINS', 'PERSONAL_CLEANSING', 'HOUSEHOLD',
#        'ORAL_HYGIENE'], dtype=object)

array(['CANDY'], dtype=object)

In [21]:
df_filtered.shape
# (34145, 6)

(6827, 7)

In [22]:
df_filtered.head()

,store_nbr,category,external_cluster_labels,internal_cluster_labels,demand_cluster_labels,rebalanced_demand_cluster_labels,CAT_DSC
0,5231,CANDY,5,2,5_2,5_2,CANDY
1,1435,CANDY,6,1,6_1,6_1,CANDY
2,9647,CANDY,3,1,3_1,3_1,CANDY
3,8945,CANDY,6,1,6_1,6_1,CANDY
4,5045,CANDY,1,1,1_1,1_1,CANDY


In [23]:
df_filtered.CAT_DSC.unique()

array(['CANDY'], dtype=object)

# Export

In [24]:
latest_datetime = earliest_datetime

In [25]:
# -------------------------------------------------------------------
# Snowflake-related imports and writing to Snowflake
# -------------------------------------------------------------------
from fsutils import run_sf_sql as rp, config, email, adls_gen2
from snowflake.connector.pandas_tools import write_pandas

# Define the config map name
configmapname = "notebook-medium"

# Get configs
params = config.get_config(configmapname)

# Get the SF connection
conn, cur = rp.get_connection(configmapname)

df = df_filtered.copy()

# 7) Form our table name using the latest date/time
#    (if you only want the date, you could parse out the YYYYMMDD)
# table_name = f"ASSORTMENT_STORE_CLUSTERS_{latest_datetime}"
table_name = "FINAL_ASSORTMENT_STORE_CLUSTERS"

# 8) Write to Snowflake
write_pandas(
    conn,
    df,
    table_name,
    database="DL_FSCA_SLFSRV",
    schema="TWA07",
    overwrite=True,
    auto_create_table=True,
)

print(f"Data written to table: {table_name}")

Inside config.py slfsvc-twa07
Inside config.py slfsvc-twa07
printing snowflake connection dic {'SF_USER_NAME': 'sa_slfsvc_fsca_app_twa_07', 'SF_ACCOUNT': 'cvsfrontstore.east-us-2.privatelink', 'SF_WAREHOUSE': 'WH_FSCA_TWA_07', 'SF_USER_ROLE': 'ROLE_FSCA_TWA07_BATCH_FR', 'SF_DB': 'CORE_FS', 'SF_AUDIT_SCHEMA': 'WORK_FS'}
printing sfparms {'SF_USER_NAME': 'sa_slfsvc_fsca_app_twa_07', 'SF_ACCOUNT': 'cvsfrontstore.east-us-2.privatelink', 'SF_WAREHOUSE': 'WH_FSCA_TWA_07', 'SF_USER_ROLE': 'ROLE_FSCA_TWA07_BATCH_FR', 'SF_DB': 'CORE_FS', 'SF_AUDIT_SCHEMA': 'WORK_FS'}
2025-04-14 16:44:00,316 - fsutils.run_sf_sql - INFO - Establishing Snowflake Connection
Getting snowflake key file from Azure Key Vault
Getting snowflake key pass from Azure Key Vault
2025-04-14 16:44:27,982 - fsutils.run_sf_sql - INFO -  Getting Snowflake Cursor
2025-04-14 16:44:27,983 - fsutils.run_sf_sql - INFO - Got the cursor
Data written to table: FINAL_ASSORTMENT_STORE_CLUSTERS


### Appending

In [26]:
master_name = "FINAL_ASSORTMENT_STORE_CLUSTERS_ARCHIVE"

In [27]:
df_archive = df.copy()
# add a column called timestamp and use this variable latest_datetime
df_archive["timestamp"] = latest_datetime

In [28]:
write_pandas(
    conn,
    df_archive,
    master_name,
    database="DL_FSCA_SLFSRV",
    schema="TWA07",
    overwrite=False,
    auto_create_table=True,
)

(True,
 1,
 6827,
 [('gqwonolcdt/file0.txt',
   'LOADED',
   6827,
   6827,
   1,
   0,
   None,
   None,
   None,
   None)])

## Adhoc

In [45]:
dighealth = pd.read_sql(
    "select * from DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250321_1842_DIGHEALTH", conn
)
snacks = pd.read_sql(
    "select * from DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250401_1553_SNACKS", conn
)
candy = pd.read_sql(
    "select * from DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250402_1949_CANDY", conn
)
final_assortment = pd.read_sql(
    "select * from DL_FSCA_SLFSRV.TWA07.FINAL_ASSORTMENT_STORE_CLUSTERS", conn
)

In [59]:
dighealth.head(2)

,store_nbr,category,external_cluster_labels,internal_cluster_labels,demand_cluster_labels,rebalanced_demand_cluster_labels,CAT_DSC
0,5231,DIGESTIVE_HEALTH,7,1,7_1,7_1,DIGESTIVE HEALTH
1,1435,DIGESTIVE_HEALTH,1,3,1_3,1_3,DIGESTIVE HEALTH


In [54]:
snacks = snacks.rename(
    columns={"STORE_NBR": "store_nbr", "CLUSTER": "rebalanced_demand_cluster_labels"}
)

In [55]:
candy.head(2)

,store_nbr,category,external_cluster_labels,internal_cluster_labels,demand_cluster_labels,rebalanced_demand_cluster_labels,CAT_DSC
0,5231,CANDY,6,0,6_0,6_0,CANDY
1,1435,CANDY,3,2,3_2,3_2,CANDY


In [56]:
final_assortment.head(2)

,store_nbr,category,external_cluster_labels,internal_cluster_labels,demand_cluster_labels,rebalanced_demand_cluster_labels,CAT_DSC
0,5231,HAIR_CARE,2,1,2_1,2_1,HAIR CARE
1,1435,HAIR_CARE,3,0,3_0,3_0,HAIR CARE


In [57]:
# Merge all DataFrames
merged_df = pd.concat([dighealth, snacks, candy, final_assortment], ignore_index=True)

In [63]:
merged_df

,store_nbr,category,external_cluster_labels,internal_cluster_labels,demand_cluster_labels,rebalanced_demand_cluster_labels,CAT_DSC
0,5231,DIGESTIVE_HEALTH,7,1,7_1,7_1,DIGESTIVE HEALTH
1,1435,DIGESTIVE_HEALTH,1,3,1_3,1_3,DIGESTIVE HEALTH
2,9647,DIGESTIVE_HEALTH,4,3,4_3,4_3,DIGESTIVE HEALTH
3,8945,DIGESTIVE_HEALTH,1,3,1_3,1_3,DIGESTIVE HEALTH
4,5045,DIGESTIVE_HEALTH,5,3,5_3,5_3,DIGESTIVE HEALTH
...,...,...,...,...,...,...,...
27301,2145,HAIR_CARE,6,0,6_0,6_0,HAIR CARE
27302,9188,HAIR_CARE,8,2,8_2,8_2,HAIR CARE
27303,11245,HAIR_CARE,3,0,3_0,3_0,HAIR CARE
27304,10892,HAIR_CARE,3,0,3_0,3_0,HAIR CARE


In [64]:
write_pandas(
    conn,
    merged_df,
    f"FINAL_ASSORTMENT_STORE_CLUSTERS_{latest_datetime}_MERGED",
    database="DL_FSCA_SLFSRV",
    schema="TWA07",
    overwrite=True,
    auto_create_table=True,
)

(True,
 1,
 27306,
 [('hvyryxemgd/file0.txt',
   'LOADED',
   27306,
   27306,
   1,
   0,
   None,
   None,
   None,
   None)])

In [38]:
f"FINAL_ASSORTMENT_STORE_CLUSTERS_{latest_datetime}_MERGED"

'FINAL_ASSORTMENT_STORE_CLUSTERS_20250409_1606_MERGED'

In [18]:
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
from fsutils import run_sf_sql as rp, config, email, adls_gen2, log

conn, cur = rp.get_connection("notebook-medium")
import pandas as pd

# Define the table names
tables = [
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250312_1859",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250313_1956",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250319_1339",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250319_1934",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250320_1843",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250321_1446",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250321_1842",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250324_2027",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250325_1347",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250325_1719",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250326_1342",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250326_1529",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250326_1717",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250326_1901",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250327_1912",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250328_1901",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250331_1351",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250331_1701",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250401_1553",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250402_1949",
    "DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_20250403_1746",
]

# Additional table with its own timestamp
table2 = ["DL_FSCA_SLFSRV.TWA07.FINAL_ASSORTMENT_STORE_CLUSTERS_ARCHIVE"]


# Define a function to preprocess timestamps
def preprocess_timestamp(timestamp):
    try:
        # If the timestamp is already in a valid format, return it
        return pd.to_datetime(timestamp)
    except:
        # Handle custom format like '20250404_1615'
        if "_" in timestamp:
            date_part, time_part = timestamp.split("_")
            formatted_timestamp = f"{date_part[:4]}-{date_part[4:6]}-{date_part[6:]} {time_part[:2]}:{time_part[2:]}:00"
            return pd.to_datetime(formatted_timestamp)
        else:
            raise ValueError(f"Unknown timestamp format: {timestamp}")


# Load data from each table, add timestamp column, and concatenate
dataframes = []

# Process tables with timestamps
for table in tables:
    # Extract the date from the table name
    date_str = table.split("_")[-2]  # Extract '20250312' part
    timestamp = pd.to_datetime(date_str, format="%Y%m%d")  # Convert to datetime

    # Query the table
    query = f"SELECT * FROM {table}"
    df = pd.read_sql(query, conn)  # Replace 'conn' with your database connection object

    # Add the timestamp column
    df["timestamp"] = timestamp

    # Append to the list of DataFrames
    dataframes.append(df)

# Concatenate all DataFrames from the first set of tables
final_df = pd.concat(dataframes, ignore_index=True)

# Process the additional table (table2) with its own timestamp
for table in table2:
    query = f"SELECT * FROM {table}"
    df_table2 = pd.read_sql(query, conn)  # Replace 'conn' with your database connection object

    # Ensure the timestamp column exists in table2
    if "timestamp" not in df_table2.columns:
        raise ValueError(f"The table {table} does not have a 'timestamp' column.")

    # Append table2 to the final DataFrame
    final_df = pd.concat([final_df, df_table2], ignore_index=True)

# Enforce consistent timestamp format
final_df["timestamp"] = final_df["timestamp"].apply(preprocess_timestamp)

In [20]:
write_pandas(
    conn,
    final_df,
    f"FINAL_ASSORTMENT_STORE_CLUSTERS_MERGED_MICHELLE_Apr_10_2025",
    database="DL_FSCA_SLFSRV",
    schema="TWA07",
    overwrite=True,
    auto_create_table=True,
)

(True,
 1,
 566401,
 [('urrclvppuv/file0.txt',
   'LOADED',
   566401,
   566401,
   1,
   0,
   None,
   None,
   None,
   None)])